---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [155]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Create dataframes for train and test set, change maximum column display length to 8, and set 'ticket id' as the index for
    # both train and test dataframe.
    df_train = pd.read_csv('train.csv', engine = 'python')
    df_test = pd.read_csv('test.csv')
    pd.set_option("display.max_columns", 8)

    # *1. Pre-processing: Clean up dataframes and format them to make it easier to interpret. 

    # (1) Set 'ticket_id' as the index, for both train and test dataframes

    df_train.set_index('ticket_id', inplace=True)
    df_test.set_index('ticket_id', inplace=True)

    # (2) For train set, drop all rows where the value of the class variable is invalid. In this assignment, drop all rows
    # where 'compliance' == NaN

    df_train.drop(df_train[df_train['compliance'].isnull()].index, inplace = True)

    # *2. Pre-Processing: Feature selection*

    # (1) Remove features directly correlated with class variable, to prevent data leakage regarding future outcomes. 
    # I.e. remove [payment_amount, payment_date, payment_status, balance_due, collection_status, compliance_detail]

    df_train = df_train.drop(['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status',
                             'compliance_detail'], axis = 1)

    # (2) Select only one feature column per feature concept to reduce redundancy.
    # Use zip code as location code of violators since it includes country, state, city, and street in a number coding

    redundant_features = ['violation_street_number', 'violation_street_name', 'mailing_address_str_number', 
                          'mailing_address_str_name', 'city', 'state', 'country', 'violation_description']
    df_train = df_train[df_train.columns[~df_train.columns.isin(redundant_features)]]

    # (3) Remove all columns where there are too many NaN values. Cutoff is > 95% of data. 

    features_with_too_many_NaN = list(df_train.isnull().sum()[df_train.isnull().sum() > (len(df_train)*0.95)].index)
    df_train = df_train[df_train.columns[~df_train.columns.isin(features_with_too_many_NaN)]]

    # (4) Assign variable types for the remaining features: categorical, ordinal, interval, ratio

    categorical = ['agency_name', 'inspector_name', 'violator_name', 'violation_code', 'disposition']
    ordinal = ['zip_code'] # zip code is considered ordinal in the sense that numbers with smaller differences represent similar
                            # locations
    interval = ['ticket_issued_date', 'hearing_date']
    ratio = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
           'clean_up_cost', 'judgment_amount', 'compliance']

    # (5) Convert the two interval variable features, {'ticket_issued_date', 'hearing_date'}, which are associated with each other 
    # as they use the same measure of time (Year-Month-Day-Hour-Minute-Second), into a single ratio feature, by subtracting
    # ticket issued date from hearing date to get the amount of passed time. 
    # i.e. amount_of_time_waited_for_hearing = hearing date in total days - ticket issued date in total days

    # Convert each hearing date to number of days from 0 a.d.

    df_train['hearing_date_numeric'] = df_train.hearing_date.str[0:10]
    df_train['hearing_date_numeric'] = df_train['hearing_date_numeric'].str.replace('-','')

    df_train['hearing_date_number_of_years'] = df_train['hearing_date_numeric'].str[0:4].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)
    df_train['hearing_date_number_of_months'] = df_train['hearing_date_numeric'].str[4:6].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)

    df_train['hearing_date_number_of_days'] = df_train['hearing_date_numeric'].str[6:8].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)

    df_train['hearing_date_total_days'] = (df_train['hearing_date_number_of_years']*365 + 
                                           df_train['hearing_date_number_of_months']*30 +
                                           df_train['hearing_date_number_of_days'])

    # Convert each ticket issue date to number of days from 0 a.d.

    df_train['ticket_issued_date_numeric'] = df_train.ticket_issued_date.str[0:10]
    df_train['ticket_issued_date_numeric'] = df_train['ticket_issued_date_numeric'].str.replace('-','')

    df_train['ticket_issued_date_number_of_years'] = df_train['ticket_issued_date_numeric'].str[0:4].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)
    df_train['ticket_issued_date_number_of_months'] = df_train['ticket_issued_date_numeric'].str[4:6].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)
    df_train['ticket_issued_date_number_of_days'] = df_train['ticket_issued_date_numeric'].str[6:8].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)

    df_train['ticket_issued_date_total_days'] = (df_train['ticket_issued_date_number_of_years']*365 + 
                                           df_train['ticket_issued_date_number_of_months']*30 +
                                           df_train['ticket_issued_date_number_of_days'])

    # Get amount_of_time_waited_for_hearing (hearing date in total days - ticket issued date in total days)
    df_train['time_until_hearing_date'] = df_train['hearing_date_total_days'] - df_train['ticket_issued_date_total_days']

    # (6) Disregard every categorical variable feature to which the number of unique categorical values is close to identical 
    # to the length of the dataset. I.e. Disregard categorical feautres to which each object row of the dataset has a near 
    # unique value for a categorical feature. E.g. I can assume that the violators would most likely have unique names because 
    # it is rare for people to have both identical first and last names. 

    # The cut-off for number of categorical values of a feature being near identical to the length of the dataset is: 
    # number of values >= 50% of dataset length.

    cut_off = len(df_train)*0.5 # min number of unique values of features that are not selected. 
    selected_categorical_features = []
    for x in categorical:
        if (len(df_train[x].unique()) < cut_off):
            selected_categorical_features.append(x)

    # (7) Change data type of ordinal variables to numbers. 

    df_train['zip_code'] = pd.to_numeric(df_train['zip_code'], errors = 'coerce')

    # (8) Remove irrelevant features from the dataframe: all irrelevant categorical variables, and interval variable features
    # which have been combined to new ratio variables. 

    selected_features = selected_categorical_features + ordinal
    selected_features.append('time_until_hearing_date')
    selected_features.extend(ratio)

    df_train = df_train[selected_features] # revised training dataframe with selected 14 features out of 26

    # (9) Convert all categorical variables to numeric form. To do this use the 'one hot encoding' method. 

    df_train_final = pd.get_dummies(df_train, columns = list(df_train.columns[df_train.dtypes == object]))

    # (10) Clean up the NaN values of the dataframe by replacing them with the mean value of that column. 

    df_train_final.fillna(df_train_final.mean(), inplace=True);
    # df_train_final.columns[df_train_final.isnull().sum() > 0]

    # *2. Selection of model and training. 
    # (1) First choose which type of model I will use for this dataset. Since the class variable are discrete and binomial,
    # i.e. binary classifiation problem, this will not use regressors, but rather classifiers. Possible classifiers to 
    # use: K-nn classifier, LSVC, KSVC, Logistic Regression, Decision Tree method, Naive Bayes.

    # Rule out K-nn classifier, because there seems to be too many features which would make the k-nn classifier too slow to 
    # perform. 

    # Rule out Naive Bayes, because there seems to be multiple features which are strongly causally dependent to each other
    # such as: agency_name & inspector_name, violation_code & disposition, fine_amount & clean_up_cost.

    # Rule out LSVC because the data is likely not linearly separable, given how unintuitive this problem seems (e.g. its 
    # not like predicting a person's weight based on height, which is such an intuitive relationship that would make it
    # linearly separable)

    # Rule out logistic regression for the same reason for ruling out LSVC

    # Rule out KSVC because of inefficiecy handling large data, and because it does not provide direct probability measurements. 
    # But because it seems to be a very flexible classification method (because I can choose which kernel to use), and because 
    # there is an indirect way to compute the probability estimates of the class, this is a second candidate model method to use.

    # Try decision tree method. If it doesn't perform well, then use KSVC. 

    # (2) Decision tree method: Random Forest. No need for feature normalization because decision trees don't perform arithmetic
    # operations between feature variables. 

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split

    # First perform a train-test split on the data on the training set. Note X_test and y_test is really validation data. 

    feature_names = list(df_train_final.columns[df_train_final.columns != 'compliance'])
    X_df_final = df_train_final[feature_names]
    y_df_final = df_train_final['compliance']

    X_train, X_test, y_train, y_test = train_test_split(X_df_final, y_df_final, random_state=0)

    # Train the RF model 

    clf = RandomForestClassifier(random_state = 0)
    clf.fit(X_train, y_train)

    # Cross-validation of the model for the evaluation metric = AUC.

    from sklearn.model_selection import cross_val_score
    cv_scores = cross_val_score(clf, X_test, y_test, cv = 10, scoring='roc_auc')

    # *2. Testing out the model

    # (1) Format the test set exactly in the way it is done for the training set. I.e. same features.
    # Convert each hearing date to number of days from 0 a.d.

    df_test['hearing_date_numeric'] = df_test.hearing_date.str[0:10]
    df_test['hearing_date_numeric'] = df_test['hearing_date_numeric'].str.replace('-','')

    df_test['hearing_date_number_of_years'] = df_test['hearing_date_numeric'].str[0:4].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)
    df_test['hearing_date_number_of_months'] = df_test['hearing_date_numeric'].str[4:6].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)

    df_test['hearing_date_number_of_days'] = df_test['hearing_date_numeric'].str[6:8].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)

    df_test['hearing_date_total_days'] = (df_test['hearing_date_number_of_years']*365 + 
                                           df_test['hearing_date_number_of_months']*30 +
                                           df_test['hearing_date_number_of_days'])

    # Convert each ticket issue date to number of days from 0 a.d.

    df_test['ticket_issued_date_numeric'] = df_test.ticket_issued_date.str[0:10]
    df_test['ticket_issued_date_numeric'] = df_test['ticket_issued_date_numeric'].str.replace('-','')

    df_test['ticket_issued_date_number_of_years'] = df_test['ticket_issued_date_numeric'].str[0:4].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)
    df_test['ticket_issued_date_number_of_months'] = df_test['ticket_issued_date_numeric'].str[4:6].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)
    df_test['ticket_issued_date_number_of_days'] = df_test['ticket_issued_date_numeric'].str[6:8].apply(lambda x: int(x) 
                                                                                               if type(x) == str else x)

    df_test['ticket_issued_date_total_days'] = (df_test['ticket_issued_date_number_of_years']*365 + 
                                           df_test['ticket_issued_date_number_of_months']*30 +
                                           df_test['ticket_issued_date_number_of_days'])

    # Get amount_of_time_waited_for_hearing (hearing date in total days - ticket issued date in total days)
    df_test['time_until_hearing_date'] = df_test['hearing_date_total_days'] - df_test['ticket_issued_date_total_days']

    df_test['zip_code'] = pd.to_numeric(df_test['zip_code'], errors = 'coerce')

    selected_features = selected_categorical_features + ordinal
    selected_features.append('time_until_hearing_date')
    test_ratio = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
           'clean_up_cost', 'judgment_amount']
    selected_features.extend(test_ratio)
    df_test = df_test[selected_features] # revised test dataframe with selected 13

    df_test_dummies = pd.get_dummies(df_test, columns = list(df_test.columns[df_test.dtypes == object]))

    # Add only the categorical dummy variable columns which is also in df_train_final to df_test

    for x in df_test_dummies.columns:
        if x not in ['zip_code','time_until_hearing_date', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
           'clean_up_cost', 'judgment_amount']: #I.e. if x is a test set categorical dummy variable
            if x in df_train_final.columns:
                df_test[x] = df_test_dummies[x] # add categorical dummy column to df_test if the column name is in df_train_final

    # Add only the categorical dummy variable columns which is only in the df_train_final to df_test, but such that 
    # df_test[x] = 0

    for x in df_train_final.columns:
        if x not in df_test_dummies.columns:
            if x != 'compliance':
                df_test[x] = 0

    # Remove columns from df_test that is not in df_train_final

    for x in df_test.columns:
        if x not in df_train_final.columns:
            df_test.drop(x, axis = 1, inplace = True)

    df_test.fillna(df_test.mean(), inplace=True)
    df_test_final = df_test

    # (2) Get the predicted probability of class values of the test set from the model
    predicted_probabilities = clf.predict_proba(df_test_final)

    # (3) As the final step of this assignment, return a Panda series with row index labels with corresponding predicted
    # probabiliies of class values of the test set.


    # First get the list of class prediction probabilities for each index. 
    single_value_prediction = []
    for x in predicted_probabilities:
        if x[0] > x[1]: # when probability of class = 0 is greater than probability of class = 1
            single_value_prediction.append(1 - x[0])
        else: # when probability of class = 1 is greater than probability of class = 0 
            single_value_prediction.append(x[1])

    df_test_final['compliance'] = single_value_prediction

    return df_test_final['compliance']

In [156]:
blight_model()

ticket_id
284932    0.300000
285362    0.000000
285361    0.300000
285338    0.250000
285346    0.350000
285345    0.350000
285347    0.200000
285342    0.800000
285530    0.000000
284989    0.300000
285344    0.200000
285343    0.000000
285340    0.000000
285341    0.200000
285349    0.300000
285348    0.200000
284991    0.100000
285532    0.100000
285406    0.000000
285001    0.100000
285006    0.100000
285405    0.000000
285337    0.100000
285496    0.100000
285497    0.200000
285378    0.100000
285589    0.100000
285585    0.300000
285501    0.400000
285581    0.100000
            ...   
376367    0.300000
376366    0.100000
376362    0.100000
376363    0.266667
376365    0.300000
376364    0.100000
376228    0.100000
376265    0.100000
376286    0.600000
376320    0.100000
376314    0.200000
376327    0.600000
376385    0.600000
376435    0.700000
376370    0.700000
376434    0.200000
376459    0.400000
376478    0.100000
376473    0.200000
376484    0.300000
376482    0.400000
37